In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from pandas.tests.frame.test_sort_values_level_as_str import ascending

import os,re,math,string,copy,json
import requests,joblib

import pickle

import gensim
from gensim import corpora

from docutils.nodes import document

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore", category = DeprecationWarning)

In [3]:
def Documents():
    
    #read the document first 
    path = '/Users/srijeetchatterjee/Desktop/Projects/ATT/Search_Engine/'
    documents = pd.read_excel(path + 'documents/Veracode-Report.xlsx')
    
    print(documents.columns)
    # drop the na rows as they do not hold any value at all 
    documents = documents.dropna()
    documents['Description'] = documents['Description'].str.strip()
    documents['Description'] = documents['Description'].str.replace('\\n', ' ')
    documents['Description'] = documents['Description'].str.replace('\\t', ' ')
    
    documents['Description'] = documents['Description'].apply(
        lambda x: " ".join(x.lower() for x in x.split()))
    
    lemmatizer_wordnet = WordNetLemmatizer()
    
    documents['Description'] = documents['Description'].apply(lambda x: " ".join(
        [lemmatizer_wordnet.lemmatize(word, pos = 'v') for word in x.split()]))
    

    tokens = [nltk.word_tokenize(token) for token in documents['Description']]
    
    
    Docdictionary = corpora.Dictionary(tokens)
#     print(type(Docdictionary))
#     print(Docdictionary)
    
    Doccorpus = [Docdictionary.doc2bow(token) for token in tokens]
    
#     print(type(Doccorpus))
#     print(Doccorpus)
    
    Doctf_idf = gensim.models.TfidfModel(Doccorpus)
#     print(type(Doctf_idf))
#     print(Doctf_idf)
    
    Docsims = gensim.similarities.Similarity(None, Doctf_idf[Doccorpus], num_features = len(Docdictionary))
#     print(type(Docsims))
#     print(Docsims)
    
    joblib.dump(Docdictionary, path+'documents/pickle/Docdictonary.pkl')
    joblib.dump(Doctf_idf, path+'documents/pickle/Doctf_idf.pkl')
    joblib.dump(Docsims, path+'documents/pickle/Docsims.pkl')

    documents.to_csv(path +'documents/pickle/vulnerable_baseline.csv')

Index(['Name', 'Severity', 'Description', 'Recommendation',
       'Associated Findings:'],
      dtype='object')


In [4]:
Documents()

Index(['Name', 'Severity', 'Description', 'Recommendation',
       'Associated Findings:'],
      dtype='object')


In [5]:
def Datacleaning(documents):
    documents['Description'] = documents['Description'].str.replace('\n', ' ')
    documents['Description'] = documents['Description'].str.replace('\xa0', ' ')
    documents['Description'] = documents['Description'].str.replace('\r', ' ')

    documents['Description'] = documents['Description'].apply(
        ''.join).str.replace(r'[^A-Za-z\s\n]+', '')

    documents['Description'] = token_lemitization(documents['Description'])

    return documents

In [6]:
def token_lemitization(Data):
    
    text_tokenised = []
    punctuation = re.compile(r'[-.?!,":;()|0-9]')
    
    for text in Data:
        text = punctuation.sub('', text)
        text_tokenised.append(word_tokenize(text.lower()))

    text_lemitized = []

    lemmatizer_wordnet = WordNetLemmatizer()

    for text_list in text_tokenised:
        word_lemitized = []
        for text in text_list:
            if text != '':
                word_lemitized.append(
                    lemmatizer_wordnet.lemmatize(text, pos='v'))
        text_lemitized.append(word_lemitized)

    text_string = []

    for text in text_lemitized:
        text_string.append(' '.join(text))
    
    return text_string

In [7]:
def queryProcessing(query):
    query = query.lower()
    query = query.replace(r'[^A-Za-z\s\n]+', '')
    lemmatizer_wordnet = WordNetLemmatizer()
    query = " ".join(lemmatizer_wordnet.lemmatize(word, pos='v')for word in query.split())
    
    return query

In [10]:
path = '/Users/srijeetchatterjee/Desktop/Projects/ATT/Search_Engine/documents/pickle/'
Docdict = joblib.load(path + 'Docdictonary.pkl')
Doctfidf = joblib.load(path + 'Doctf_idf.pkl')
Doctsimilarity = joblib.load(path + 'Docsims.pkl')
Docdoc = pd.read_csv(path + 'vulnerable_baseline.csv', na_filter=False)


def vulDocSearch(query):
    
    query = queryProcessing(query)
    
    token_prob = nltk.word_tokenize(query)
    
    # tokenized user query is matched with the pickle file(which is bow) and creatinga doc2bow dictionary.
    doc2bow = Docdict.doc2bow(token_prob)
#     print(f"doc2bow : {doc2bow}")
#     print('\n')

    # dictionary created is converted into tfidf.
    doc_tfidf = Doctfidf[doc2bow]
#     print(f"doc_tfidf : {doc_tfidf}")
#     print('\n')
    
    # taking similarity of user query from the Doctsimilarity.pkl
    similarity = Doctsimilarity[doc_tfidf]
#     print(f"similarity : {similarity}")
#     print('\n')
    
    # sorting similarity in descending order
    indices = similarity.argsort()[::-1]
    
    dde = Docdoc.iloc[indices]
    
    score = similarity[indices]
    dde['Prediction%'] = np.round(score*100)  # to get the % in the /100 format
    
    dde = dde.loc[dde['Prediction%'] > 0]
    dde = dde[0:75]

    # selecting the fields which we want to show on the screen.
    dde = dde[['Prediction%', 'Name','Description', 'Recommendation', 'Severity']]
    
    return dde

In [11]:
query = input()
print('The searched results are : ')
print(vulDocSearch(query))

SQL injection vulnerabilities occur This allows an attacker to
The searched results are : 
   Prediction%                           Name  \
0         38.0                  SQL Injection   
3          7.0                 CRLF Injection   
2          5.0           Cross-Site Scripting   
1          3.0            Directory Traversal   
5          3.0  Insufficient Input Validation   

                                         Description  \
0  sql injection vulnerabilities occur when data ...   
3  the acronym crlf stand for "carriage return, l...   
2  cross-site script (xss) attack occur when an a...   
1  allow user input to control paths use in files...   
5  weaknesses in this category be relate to an ab...   

                                      Recommendation Severity  
0  Several techniques can be used to prevent SQL ...     High  
3  Apply robust input filtering for all user-supp...   Medium  
2  Several techniques can be used to prevent XSS ...   Medium  
1  Assume all user-su